# Out-of-State-Contributions: Candidates Analysis

In [1]:
from functools import reduce
import numpy as np
import pandas as pd

%load_ext jupyternotify

pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 50)
pd.options.display.float_format = "{:,.2f}".format

<IPython.core.display.Javascript object>

Import [National Institute on Money in Politics](https://www.followthemoney.org/) API key.

In [2]:
nimp_key = open("nimp_api_key.txt", "r").readline()

## Question: How much out-of-state money are candidates raising in the 2018 election cycle, in absolute and proportional terms, thus far and how does that compare with the 2014 and 2010 cycles?

### Import and format contribution-level data on donations to gubernatorial, attorney general, secretary of state, state supreme court, state senate and state house candidates in 2018, 2014 and 2010.

Our first task is to determine a data cut-off point for prior election cycles so we can make accurate comparisons across cycles.

Download and save each cycle's contributions data.

In [ ]:
#%%notify
#%%time
#contributions_18 = pd.read_csv("https://www.followthemoney.org/aaengine/aafetch.php?dt=1&y=2018&c-exi=1&c-r-oc=Z10,Z70&c-r-ot=G,S,H,J&gro=c-t-id,d-id&APIKey="+nimp_key+"&mode=csv")
#contributions_18.to_csv("data/contributions_18.csv", index=False)
#contributions_14 = pd.read_csv("https://www.followthemoney.org/aaengine/aafetch.php?dt=1&y=2014&c-exi=1&c-r-oc=Z10,Z70&c-r-ot=G,S,H,J&gro=c-t-id,d-id&APIKey="+nimp_key+"&mode=csv")
#contributions_14.read_csv("data/contributions_14.csv", index=False)
#contributions_10 = pd.read_csv("https://www.followthemoney.org/aaengine/aafetch.php?dt=1&y=2010&c-exi=1&c-r-oc=Z10,Z70&c-r-ot=G,S,H,J&gro=c-t-id,d-id&APIKey="+nimp_key+"&mode=csv")
#contributions_10.read_csv("data/contributions_10.csv", index=False)

Concatenate the data.

In [3]:
%%bash
head -1 "data/raw/contributions_18.csv" >> "data/raw/contributions.csv"
sed '1d' "data/raw/contributions_18.csv" >> "data/raw/contributions.csv"
sed '1d' "data/raw/contributions_14.csv" >> "data/raw/contributions.csv"
sed '1d' "data/raw/contributions_10.csv" >> "data/raw/contributions.csv"

Import the data.

In [4]:
contributions = pd.read_csv("data/raw/contributions.csv", usecols=["Candidate", "Election_Jurisdiction", "Election_Year", "Office_Sought", "Contributor", "Amount", "Date", "In-State"], error_bad_lines=False)
contributions.columns = ["candidate", "state", "year", "office", "contributor", "amount", "date", "in_out_state"]
contributions.head(1)

/home/jyerardi/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (54) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,candidate,state,year,office,contributor,amount,date,in_out_state
0,"RAUNER, BRUCE VINCENT & SANGUINETTI, EVELYN PA...",IL,2018,GOVERNOR / LIEUTENANT GOVERNOR,"RAUNER, BRUCE VINCENT","50,000,000.00",2016-12-20,1.00


Delete the data.

In [7]:
%%bash
rm "data/raw/contributions_18.csv"
rm "data/raw/contributions_14.csv"
rm "data/raw/contributions_10.csv"
rm "data/raw/contributions.csv"

Convert the contribution amount column to numeric (float) data type and convert the contribution date column to datetime data type.

In [8]:
contributions["amount"] = pd.to_numeric(contributions["amount"], errors="coerce")
contributions["date"] = pd.to_datetime(contributions["date"], errors="coerce")

Filter out unitemized donations as it is impossible to determine where those contributions originated.

In [9]:
contributions = contributions[contributions["contributor"] != "UNITEMIZED DONATIONS"]

Rename the categories in the in-vs.-out-of-state column.

In [10]:
# 0 = out-of-state, 1 = in-state, 2 = unknown
contributions["in_out_state"] = contributions["in_out_state"].replace({0: "out-of-state", 1: "in-state", 2: "unknown"})

Create a standardized office column.

In [11]:
contributions["standardized_office"] = np.where(contributions["office"].str.contains("governor", case=False), "GOVERNOR/LIEUTENANT GOVERNOR",
                              np.where(contributions["office"].str.contains("attorney general", case=False), "ATTORNEY GENERAL",
                              np.where(contributions["office"].str.contains("secretary of state", case=False), "SECRETARY OF STATE",
                              np.where(contributions["office"].str.contains("court", case=False), "SUPREME COURT",
                              np.where(contributions["office"].str.contains("senate", case=False), "STATE SENATE",
                              np.where(contributions["office"].str.contains("house", case=False), "STATE HOUSE", ""))))))

contributions.head(1)

,candidate,state,year,office,contributor,amount,date,in_out_state,standardized_office
0,"RAUNER, BRUCE VINCENT & SANGUINETTI, EVELYN PA...",IL,2018,GOVERNOR / LIEUTENANT GOVERNOR,"RAUNER, BRUCE VINCENT","50,000,000.00",2016-12-20,in-state,GOVERNOR/LIEUTENANT GOVERNOR


### Calculate each state's data cut-off point for the 2018 cycle's data.

Filter the data to just 2018 cycle contributions. Then extract the month and year from the contribution date column.

In [12]:
contributions_18 = contributions[contributions["year"] == 2018]
contributions_18["month"] = contributions_18["date"].dt.to_period("M")
contributions_18.info()

/home/jyerardi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2150964 entries, 0 to 2180762
Data columns (total 10 columns):
candidate              object
state                  object
year                   int64
office                 object
contributor            object
amount                 float64
date                   datetime64[ns]
in_out_state           object
standardized_office    object
month                  object
dtypes: datetime64[ns](1), float64(1), int64(1), object(7)
memory usage: 180.5+ MB


Group the contributions by state and month.

In [13]:
grouped_by_month = contributions_18.groupby(["state", "month"])["amount"].sum().reset_index()
grouped_by_month.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1489 entries, 0 to 1488
Data columns (total 3 columns):
state     1489 non-null object
month     1489 non-null object
amount    1489 non-null float64
dtypes: float64(1), object(2)
memory usage: 35.0+ KB


Because we eventually want to use each state's month column as the cut-off date for contributions, we need to add a day to the month and the year and then convert the column into datetime data type.

In [14]:
grouped_by_month["month"] = grouped_by_month["month"].astype(str) + "-28" # No month has fewer than 28 days
grouped_by_month["month"] = pd.to_datetime(grouped_by_month["month"], errors="coerce")
grouped_by_month.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1489 entries, 0 to 1488
Data columns (total 3 columns):
state     1489 non-null object
month     1489 non-null datetime64[ns]
amount    1489 non-null float64
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 35.0+ KB


We know some of the contribution dates are wrong. We know this because some of the dates occur in the future and, unless we've got some time travelling campaign donors, these are data entry errors. To eliminate this noise, we will filter out months after August 2018.

In [15]:
grouped_by_month = grouped_by_month[grouped_by_month["month"] <= "2018-08-28"]

Return the most recent month of contributions for each state.

In [16]:
latest_month = grouped_by_month.groupby("state")["month"].max().reset_index()
latest_month.rename(columns={"month": "latest_month"}, inplace=True)
latest_month

,state,latest_month
0,AK,2018-07-28
1,AL,2018-07-28
2,AR,2018-03-28
3,AZ,2017-12-28
4,CA,2018-07-28
5,CO,2018-07-28
6,CT,2018-07-28
7,DE,2017-11-28
8,FL,2018-08-28
9,GA,2018-07-28


### Apply the cut-off date to the 2014 and 2010 election cycles' data.

Filter the data by election cycle.

In [27]:
contributions_14 = contributions[contributions["year"] == 2014]
contributions_10 = contributions[contributions["year"] == 2010]

Join the table of the 2018 cycle's latest contribution months with the 2014 and 2010 contribution-level data.

In [28]:
contributions_14 = contributions_14.merge(latest_month, on="state")
contributions_10 = contributions_10.merge(latest_month, on="state")

Convert the year in the latest month column to its equivalent in the relevant election cycle.

In [29]:
# 2017 = 2013, 2018 = 2014
contributions_14["latest_month"] = contributions_14["latest_month"].mask(contributions_14["latest_month"].dt.year == 2017, contributions_14["latest_month"] + pd.offsets.DateOffset(year=2013))
contributions_14["latest_month"] = contributions_14["latest_month"].mask(contributions_14["latest_month"].dt.year == 2018, contributions_14["latest_month"] + pd.offsets.DateOffset(year=2014))
# 2017 = 2009, 2018 = 2010
contributions_10["latest_month"] = contributions_10["latest_month"].mask(contributions_10["latest_month"].dt.year == 2017, contributions_10["latest_month"] + pd.offsets.DateOffset(year=2009))
contributions_10["latest_month"] = contributions_10["latest_month"].mask(contributions_10["latest_month"].dt.year == 2018, contributions_10["latest_month"] + pd.offsets.DateOffset(year=2010))

/home/jyerardi/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/datetimes.py:962: PerformanceWarning: Non-vectorized DateOffset being applied to Series or DatetimeIndex
  "or DatetimeIndex", PerformanceWarning)


Filter the data to eliminate contributions after the 2018 cycle's latest contribution month in each state.

In [30]:
contributions_14 = contributions_14[contributions_14["date"] <= contributions_14["latest_month"]]
contributions_10 = contributions_10[contributions_10["date"] <= contributions_10["latest_month"]]

### Calculate out-of-state contributions

In [31]:
contributions_by_candidate_18 = contributions_18.groupby(["candidate", "state", "year", "standardized_office", "in_out_state"])["amount"].sum().reset_index()
contributions_by_candidate_18.rename(columns={"amount": "amount_18"}, inplace=True)
contributions_by_candidate_14 = contributions_14.groupby(["candidate", "state", "year", "standardized_office", "in_out_state"])["amount"].sum().reset_index()
contributions_by_candidate_14.rename(columns={"amount": "amount_14"}, inplace=True)
contributions_by_candidate_10 = contributions_10.groupby(["candidate", "state", "year", "standardized_office", "in_out_state"])["amount"].sum().reset_index()
contributions_by_candidate_10.rename(columns={"amount": "amount_10"}, inplace=True)

Pivot dataframe to aggregate each candidate's data in a single row.

In [32]:
contributions_by_candidate_18 = pd.pivot_table(contributions_by_candidate_18, index=["candidate", "state", "year", "standardized_office"], columns=["in_out_state"]).reset_index()
contributions_by_candidate_14 = pd.pivot_table(contributions_by_candidate_14, index=["candidate", "state", "year", "standardized_office"], columns=["in_out_state"]).reset_index()
contributions_by_candidate_10 = pd.pivot_table(contributions_by_candidate_10, index=["candidate", "state", "year", "standardized_office"], columns=["in_out_state"]).reset_index()

Some records have no contributions for certain categories. Let's set those values equal to zero to be sure any calculations we run on them are correct.

In [33]:
contributions_by_candidate_18.fillna(0, inplace=True)
contributions_by_candidate_14.fillna(0, inplace=True)
contributions_by_candidate_10.fillna(0, inplace=True)

Flatten the resulting dataframes' multi-index columns.

In [34]:
contributions_by_candidate_18.columns = ["_".join(column).replace("-","_").strip("_") for column in contributions_by_candidate_18.columns.values]
contributions_by_candidate_14.columns = ["_".join(column).replace("-","_").strip("_") for column in contributions_by_candidate_14.columns.values]
contributions_by_candidate_10.columns = ["_".join(column).replace("-","_").strip("_") for column in contributions_by_candidate_10.columns.values]

In [35]:
contributions_by_candidate_18.head(1)

,candidate,state,year,standardized_office,amount_18_in_state,amount_18_out_of_state,amount_18_unknown
0,"ABBOTT, DAVID H",IN,2018,STATE HOUSE,"26,065.00",0.00,0.00


In [36]:
contributions_by_candidate_18.rename(columns={"standardized_office": "standardized_office_18"}, inplace=True)
contributions_by_candidate_14.rename(columns={"standardized_office": "standardized_office_14"}, inplace=True)
contributions_by_candidate_10.rename(columns={"standardized_office": "standardized_office_10"}, inplace=True)

Calculate the proportion of in-state, out-of-state and unknown contributions.

In [37]:
contributions_by_candidate_18["pct_18_in_state"] = contributions_by_candidate_18["amount_18_in_state"] / (contributions_by_candidate_18["amount_18_in_state"] + contributions_by_candidate_18["amount_18_out_of_state"] + contributions_by_candidate_18["amount_18_unknown"])
contributions_by_candidate_18["pct_18_out_of_state"] = contributions_by_candidate_18["amount_18_out_of_state"] / (contributions_by_candidate_18["amount_18_in_state"] + contributions_by_candidate_18["amount_18_out_of_state"] + contributions_by_candidate_18["amount_18_unknown"])
contributions_by_candidate_18["pct_18_unknown"] = contributions_by_candidate_18["amount_18_unknown"] / (contributions_by_candidate_18["amount_18_in_state"] + contributions_by_candidate_18["amount_18_out_of_state"] + contributions_by_candidate_18["amount_18_unknown"])
contributions_by_candidate_14["pct_14_in_state"] = contributions_by_candidate_14["amount_14_in_state"] / (contributions_by_candidate_14["amount_14_in_state"] + contributions_by_candidate_14["amount_14_out_of_state"] + contributions_by_candidate_14["amount_14_unknown"])
contributions_by_candidate_14["pct_14_out_of_state"] = contributions_by_candidate_14["amount_14_out_of_state"] / (contributions_by_candidate_14["amount_14_in_state"] + contributions_by_candidate_14["amount_14_out_of_state"] + contributions_by_candidate_14["amount_14_unknown"])
contributions_by_candidate_14["pct_14_unknown"] = contributions_by_candidate_14["amount_14_unknown"] / (contributions_by_candidate_14["amount_14_in_state"] + contributions_by_candidate_14["amount_14_out_of_state"] + contributions_by_candidate_14["amount_14_unknown"])
contributions_by_candidate_10["pct_10_in_state"] = contributions_by_candidate_10["amount_10_in_state"] / (contributions_by_candidate_10["amount_10_in_state"] + contributions_by_candidate_10["amount_10_out_of_state"] + contributions_by_candidate_10["amount_10_unknown"])
contributions_by_candidate_10["pct_10_out_of_state"] = contributions_by_candidate_10["amount_10_out_of_state"] / (contributions_by_candidate_10["amount_10_in_state"] + contributions_by_candidate_10["amount_10_out_of_state"] + contributions_by_candidate_10["amount_10_unknown"])
contributions_by_candidate_10["pct_10_unknown"] = contributions_by_candidate_10["amount_10_unknown"] / (contributions_by_candidate_10["amount_10_in_state"] + contributions_by_candidate_10["amount_10_out_of_state"] + contributions_by_candidate_10["amount_10_unknown"])

Join the 2018, 2014 and 2010 contributions by candidate data

In [38]:
list_of_contributions_by_candidate = [contributions_by_candidate_18, contributions_by_candidate_14, contributions_by_candidate_10]
contributions_by_candidate = reduce(lambda left, right: pd.merge(left, right, on=["candidate", "state"], how="outer"), list_of_contributions_by_candidate)
contributions_by_candidate.drop(["year_x", "year_y", "year"], axis=1, inplace=True)
contributions_by_candidate.head()

,candidate,state,standardized_office_18,amount_18_in_state,amount_18_out_of_state,amount_18_unknown,pct_18_in_state,pct_18_out_of_state,pct_18_unknown,standardized_office_14,amount_14_in_state,amount_14_out_of_state,amount_14_unknown,pct_14_in_state,pct_14_out_of_state,pct_14_unknown,standardized_office_10,amount_10_in_state,amount_10_out_of_state,amount_10_unknown,pct_10_in_state,pct_10_out_of_state,pct_10_unknown
0,"ABBOTT, DAVID H",IN,STATE HOUSE,"26,065.00",0.00,0.00,1.00,0.00,0.00,NaN,nan,nan,nan,nan,nan,nan,NaN,nan,nan,nan,nan,nan,nan
1,"ABBOTT, GHERT",AK,STATE HOUSE,45.90,0.00,0.00,1.00,0.00,0.00,NaN,nan,nan,nan,nan,nan,nan,NaN,nan,nan,nan,nan,nan,nan
2,"ABBOTT, GREG",TX,GOVERNOR/LIEUTENANT GOVERNOR,"61,189,628.95","4,590,344.58","1,020.00",0.93,0.07,0.00,GOVERNOR/LIEUTENANT GOVERNOR,"22,988,410.36","1,488,413.64","4,791,017.12",0.79,0.05,0.16,ATTORNEY GENERAL,"7,447,937.18","570,824.87",0.00,0.93,0.07,0.00
3,"ABDUL-RAHIM, ANEES",MD,STATE HOUSE,"8,841.07",0.00,0.00,1.00,0.00,0.00,NaN,nan,nan,nan,nan,nan,nan,NaN,nan,nan,nan,nan,nan,nan
4,"ABERCROMBIE, CATHERINE F",CT,STATE HOUSE,550.00,0.00,0.00,1.00,0.00,0.00,STATE HOUSE,"4,625.00",0.00,20.00,1.00,0.00,0.00,NaN,nan,nan,nan,nan,nan,nan


Export the data to Excel.

In [39]:
contributions_by_candidate.to_excel("data/analyzed/contributions_by_candidate.xlsx", index=False)